In [3131]:
# Python 3.11.2
# %pip install -r requriments.txt

# Python 3.10.10
# %conda install --file requriments.txt
# %conda create --name PD --file file.txt

In [3132]:
# Импорт библиотек для работы с данными
import pandas as pd
import numpy as np
import scipy.stats as stats

# Импорт библиотек для визуализации данных
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Импорт библиотек для пред обработки данных
from sklearn.preprocessing import MaxAbsScaler, RobustScaler

# Импорт библиотек для отбора признаков
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SelectFpr, chi2

# Импорт библиотек линейной регрессии
import statsmodels.api as sm
from statsmodels.regression.recursive_ls import RecursiveLS
from statsmodels.regression.linear_model import OLS, GLS, GLSAR, WLS
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor
from statsmodels.discrete.discrete_model import Poisson

# Импорт библиотек для машинного обучения
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Импорт библиотек для кросс валидации
from sklearn.model_selection import train_test_split

# Подготовка данных

## Загрузка данных

In [3133]:
# Загрузка данных с помощью pandas из файла csv
data = pd.read_csv("input/data.csv", delimiter=";")
year = data['Год']

# Просмотр первых 5 строк данных
data.head()

,Год,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,1998,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1,1999,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2,2000,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
3,2001,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
4,2002,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


### Объяснение переменных
- $y_1$ - инвестиции в основной капитал, млн руб.
- $y_2$ - валовой региональный продукт (ВРП), млн.руб.
- $y_3$ - сумма доходов населения за год, млн руб.
- $x_1$ - финансовый результат деятельности (чистая прибыль)
- $x_2$ - прямые иностранные инвестиции, млн USD
- $x_3$ - среднегодовая численности занятых, тыс чел.
- $x_4$ - стоимость основных фондов, млн. руб
- $x_5$ - степень износа основных фондов, %
- $x_6$ - затраты на научные исследования и разработки, млн руб.
- $x_7$ - объём инновационных товаров работ услуг, млн руб.
- $x_8$ - экспорт, млн USD
- $x_9$ - импорт, млн. USD
- $x_{10}$ - сумма остатков вкладов на счетах в Банке России, млн. руб.
- $x_{11}$ - прожиточный минимум в регионе РФ (г. Москва), тыс.руб.

In [3134]:
class InterpolateData():
    def __init__(self, data, method='linear', freq='Q'):
        self.data = data.copy()
        self.data_quarterly = data.copy()
        self.data_quarterly['Год'] = pd.to_datetime(self.data_quarterly['Год'].astype(str), format='%Y')
        self.data_quarterly.set_index('Год', inplace=True)
        self.data_quarterly = self.data_quarterly.resample(freq).mean().interpolate(method=method)

    def head(self):
        return self.data_quarterly.head()

    def scatter(self):
        return px.scatter(self.data_quarterly)

    def original_scatter(self):
        return px.scatter(self.data.drop(columns=['Год']))

    def after_interpolate(self):
        return self.data_quarterly

In [3135]:
# InterpolateData(data).scatter()
# InterpolateData(data, method='cubic').scatter()
# InterpolateData(data, method='quadratic').scatter()
InterpolateData(data, method='akima').scatter()

In [3136]:
InterpolateData(data).original_scatter()

In [3137]:
data = InterpolateData(data, method='akima').after_interpolate()
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-03-31,87211.830000,328497.900000,638450.600000,228548.400000,803.200000,5780.870000,1.212184e+06,33.800000,83359.410000,25719.200000,6171.000000,6642.000000,111458.000000,1393.300000
1998-06-30,94837.309463,411892.838304,674544.769921,293398.535166,736.807795,5791.387033,1.217777e+06,34.608070,84434.697999,25523.220133,6463.512671,6918.993559,124811.328689,1499.653740
1998-09-30,102889.910331,502956.211813,712651.354330,358916.449175,715.808030,5802.012446,1.231037e+06,35.205168,85548.268093,25385.183334,6737.509484,7214.574151,140986.309803,1604.366468
1998-12-31,111150.973194,598751.486752,751524.460148,424389.258043,735.228912,5812.630376,1.250140e+06,35.444336,86686.056887,25300.419162,7001.119718,7517.765363,161117.650948,1707.432327
1999-03-31,119302.330000,695059.800000,789466.920000,488395.100000,787.600000,5823.010000,1.272744e+06,35.200000,87820.240000,25262.150000,7260.000000,7814.000000,185861.000000,1807.800000


## Изучение данных на наличие ошибок измерений

Функция combine_scatter_plots строит графики рассеяния для каждой пары признаков в наборе данных.

In [3138]:
def combine_scatter_plots(height=1000, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Scatter(y=data[f'y{i + 1}'], x=year, name=column), row=row, col=col)
        else:
            fig.add_trace(go.Scatter(y=data[f'x{i - 2}'], x=year, name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Зависимость признаков от времени")
    fig.show()

In [3139]:
combine_scatter_plots()

## Проверка данных на наличие выбросов и изучение распределения данных

Функция combine_box_plots строит ящиковые диаграммы для каждого признака в наборе данных, где значения признака разбиваются на квартили и представляются в виде ящика.

Функция combine_violin_plots строит аналогичные диаграммы, но вместо ящика используется график, который представляет распределение значений признака с помощью ядерной оценки плотности.

In [3140]:
# Построение графиков
def combine_box_plots(height=1000, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Box(x=data[f'y{i + 1}'], name=column), row=row, col=col)
        else:
            fig.add_trace(go.Box(x=data[f'x{i - 2}'], name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Ящиковые диаграммы распределения признаков",
                      showlegend=False)
    fig.show()


def combine_violin_plots(height=1500, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Violin(x=data[f'y{i + 1}'], name=column), row=row, col=col)
        else:
            fig.add_trace(go.Violin(x=data[f'x{i - 2}'], name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Распределение признаков", showlegend=False)
    fig.show()

In [3141]:
combine_box_plots()

In [3142]:
# combine_violin_plots()

## Избавление от мультиколлинеарности между признаками путем нормализации и центрирования данных

In [3143]:
# Центрирование данных для избавления от мультиколлинеарности
# data = data.apply(lambda x: x - x.mean())

# Минимаксная нормализация данных
data = data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Нормализация средним (Z-нормализация)
# data = data.apply(lambda x: (x - x.mean()) / x.std())

# MaxAbsScaler
# transformer = MaxAbsScaler().fit(data)
# data = pd.DataFrame(transformer.transform(data))

# RobustScaler
# transformer = RobustScaler().fit(data)
# data = pd.DataFrame(transformer.transform(data))

data.columns = ['y1', 'y2', 'y3', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11']
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-03-31,0.000000,0.000000,0.000000,0.000000,0.000549,0.000000,0.000000,0.195697,0.000000,0.001083,0.002159,0.000000,0.000000,0.000000
1998-06-30,0.001461,0.003661,0.002672,0.007145,0.000132,0.012626,0.000074,0.242208,0.002618,0.000874,0.003168,0.002142,0.000315,0.006140
1998-09-30,0.003004,0.007658,0.005493,0.014364,0.000000,0.025383,0.000250,0.276576,0.005330,0.000727,0.004113,0.004428,0.000697,0.012186
1998-12-31,0.004587,0.011863,0.008371,0.021578,0.000122,0.038130,0.000503,0.290342,0.008100,0.000637,0.005022,0.006773,0.001172,0.018136
1999-03-31,0.006149,0.016090,0.011180,0.028630,0.000451,0.050591,0.000802,0.276278,0.010861,0.000596,0.005915,0.009064,0.001756,0.023931


## Добавление шума в данные

In [3144]:
# px.scatter(data, title="Зависимость признаков от времени", height=500)
px.scatter(data['y1'], title="Зависимость признаков от времени", height=500)

In [3145]:
noise = np.random.normal(0, .06, data.shape)
noisy_data = data + noise
data = noisy_data
data = data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
px.scatter(data['y1'], title="Зависимость признаков от времени", height=500)

In [3146]:
combine_scatter_plots()

# Матрица коэффициентов межфакторной корреляции

In [3147]:
fig = px.imshow(data.corr(), text_auto='.2f', color_continuous_scale="gnbu")
fig.update_layout(height=1000, title_text="Матрица коэффициентов межфакторной корреляции")
fig.show()

In [3148]:
# un_data = data.copy()
# un_data.columns = ['Инвестиции в основной капитал, млн руб. y1', 'Валовой региональный продукт (ВРП), млн.руб. y2',
#                    'Сумма доходов населения за год, млн руб. y3',
#                    'Финансовый результат деятельности (чистая прибыль), млн.руб. x1',
#                    'Прямые иностранные инвестиции, млн USD x2',
#                    'Среднегодовая численности занятых, тыс чел. x3', 'Стоимость основных фондов, млн. руб. x4',
#                    'Степень износа основных фондов, % x5',
#                    'Затраты на научные исследования и разработки, млн руб. x6',
#                    'Объём инновационных товаров работ услуг, млн руб. x7', 'Экспорт, млн USD x8',
#                    'Импорт, млн. USD x9', 'Сумма остатков вкладов на счетах в Банке России, млн. руб. x10',
#                    'Прожиточный минимум в регионе РФ (г. Москва), тыс.руб. x11']
#
# fig, ax = plt.subplots(figsize=(10, 8))
# sns.heatmap(un_data.corr(), annot=True, fmt=".2f", linewidth=.5, cmap='cubehelix')

# Исследование данных на наличие выбросов и создание на тестовой и обучающей выборки

Нахождение выбросов - это один из методов анализа данных, который используется для выявления аномальных значений в наборе данных. Выбросы могут возникать из-за ошибок в данных, случайных вариаций или из-за наличия редких, но реальных экстремальных значений.

Существует несколько методов нахождения выбросов, в том числе метод Z-оценки и метод межквартильного размаха (IQR).

In [3149]:
# Смотрим размерность данных
data.shape

(97, 14)

Изначально мы имели набор данных, состоящий из 25 значений, охватывающих период с 1998 по 2022 год. На это нам указывает размерность данных (25, 14).

**Z-оценка**

Метод Z-оценки основан на расчете Z-оценки для каждого значения в наборе данных и проверке, насколько далеко каждое значение от среднего значения. Значения, которые находятся на расстоянии больше чем три стандартных отклонения от среднего, могут быть считаны выбросами.

In [3150]:
z = np.abs(stats.zscore(data))
data_clean = data[(z < 3).all(axis=1)]
data_clean.shape

(96, 14)

**Межквартильный размах (IQR)**

Метод межквартильного размаха (IQR) основан на вычислении интерквартильного размаха - разницы между 75-перцентилью и 25-перцентилем. Затем определяется верхняя и нижняя границы выбросов, которые определяются как 1,5 межквартильных размаха за пределами верхнего и нижнего квартилей.

In [3151]:
# Находим Первый квартиль (Q1) и Третий квартиль (Q3) и рассчитываем межквартильный размах
Q1 = data.quantile(q=.25)
Q3 = data.quantile(q=.75)
IQR = data.apply(stats.iqr)

# Оставляем только те значения, которые больше нижней границы и меньше верхней границы.
data_clean = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]
data_clean.shape

(94, 14)

В результате проверки на наличие выбросов мы получили исходный набор данных, состоящий из 25 значений, охватывающих период с 1998 по 2022 год. На это нам указывает размерность данных (25, 14). Это говорит нам о том что выбросов в данных найденно не было.

In [3152]:
# Заменяем исходный набор данных на очищенный
data = data_clean

### Создание на тестовой и обучающей выборки

In [3153]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['y1', 'y2', 'y3'], axis=1), data[['y1', 'y2', 'y3']],
                                                    test_size=0.30, random_state=42)

In [3154]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((65, 11), (29, 11), (65, 3), (29, 3))

# Построение модели линейной регрессии

Линейная регрессия - это метод анализа данных, который используется для описания и прогнозирования линейной связи между зависимой и одной или несколькими независимыми переменными. Он часто используется в статистическом анализе данных для построения математической модели, которая может быть использована для прогнозирования значений зависимой переменной на основе значений независимых переменных.

Для построения модели линейной регрессии необходимо иметь набор данных, который состоит из пар значений зависимой и независимой переменных. Затем на основе этих данных проводится анализ, который позволяет оценить параметры модели - коэффициенты наклона и пересечения. Эти параметры могут быть использованы для построения уравнения линейной регрессии.

Для оценки качества модели линейной регрессии используются различные статистические показатели, такие как коэффициент детерминации (R-квадрат) или корреляционный коэффициент. Они позволяют оценить, насколько хорошо модель соответствует данным и насколько точно она может использоваться для прогнозирования значений зависимой переменной.

Важно понимать, что линейная регрессия является моделью, которая описывает только линейные отношения между переменными. Если связь между переменными не является линейной, то модель линейной регрессии может оказаться неэффективной и требовать использования других методов анализа данных.

In [3155]:
class CustomLinearRegression:
    def __init__(self, y, x, model_type=OLS):
        self.model_type = model_type
        self.y = y
        self.x = x
        self.model = self.model_type(self.y, self.x).fit()

    def get_coefficients(self):
        print(self.model.params)
        answer = []
        names = self.x.columns.values.tolist()
        for i, par in enumerate(self.model.params):
            answer.append(f'+ {par:.4f}{names[i][0]}_{{{names[i][1:]}}}')
        print(f'$$\hat {self.y.name[0]}_{self.y.name[1]} = {" ".join(answer)[2:]}$$')

    def summary(self):
        print(self.model.summary())

    def pre(self, X_test):
        return self.model.predict(X_test)

    def predict(self, X_test, y_test):
        r2 = r2_score(y_test, self.model.predict(X_test))
        mae = mean_absolute_error(y_test, self.model.predict(X_test))
        mse = mean_squared_error(y_test, self.model.predict(X_test))
        print(f'Коэффициент детерминации (R^2): {r2 * 100:.1f}%\nCредняя ошибка аппроксимации (MAE): {mae * 100:.1f}%\nCредняя квадратичная ошибка (MSE): {mse * 100:.1f}%')


    def F(self):
        return self.model.fvalue

    def current_p(self, index):
        return self.model.f_pvalue[index]

    def forward_selection(self):
        xi = []
        stop = False
        x_new = []  # список переменных, которые будут включены в модель
        x_len = len(self.x.columns) - 1  # количество столбцов в датафрейме
        x_test = []
        for n in range(x_len):
            F_max = 0  # максимальное значение F
            p_max = 0  # максимальное значение p
            F_max_i = 0
            F_max_x = ''
            if not stop:
                for i in range(x_len):
                    if i not in xi:
                        x_test = x_new.copy()
                        x_test.append(self.x.columns[i])
                        F = CustomLinearRegression(self.y, self.x.loc[:, x_test]).F()
                        print(f"'x{i + 1}',", F)
                        if F > F_max:
                            F_max = F
                            F_max_i = i
                            F_max_x = f"x{i + 1}"
                x_new.append(F_max_x)
                xi.append(F_max_i)
                print(x_new)
                for j in range(len(x_new)):
                    p = CustomLinearRegression(self.y, self.x.loc[:, x_new]).current_p(j)
                    if p > 0.05:
                        stop = True
                        print(f'P-value: {p}')
                        break
        CustomLinearRegression(self.y, self.x.loc[:, x_new[:-1]]).summary()

    def backward_elimination(self, p=0.05, start=0):
        regression = self.model_type(self.y, self.x).fit()
        futures_num = len(self.x.columns)
        for i in range(futures_num):
            regression = self.model_type(self.y, self.x).fit()
            max_p = max(regression.pvalues.values[start:])
            if max_p > p:
                for j in range(0, futures_num - i):
                    if regression.pvalues[j] == max_p:
                        self.x = self.x.drop(self.x.columns[[j]], axis=1)
        print(regression.summary())

    def custom_feature_selection(self, x, y, model=LinearRegression(), selection_method=RFE):
        result_list = list()
        result = selection_method(model).fit(x, y)
        for i, feature in enumerate(result.get_support()):
            if feature:
                result_list.append(f'x{i + 1}')
        CustomLinearRegression(y, x[result_list]).summary()

    def correlation_map(self):
        return px.imshow(self.x.corr(), text_auto='.2f', color_continuous_scale="gnbu")


class CustomLinearRegressionWithConst(CustomLinearRegression):
    def __init__(self, y, x, model_type=OLS):
        super().__init__(y, x, model_type)
        self.x = sm.add_constant(self.x)
        self.model = self.model_type(self.y, self.x).fit()

    def backward_elimination(self, p=0.05, start=1):
        regression = self.model_type(self.y, self.x).fit()
        futures_num = len(self.x.columns)
        for i in range(futures_num):
            self.x = sm.add_constant(self.x)
            regression = self.model_type(self.y, self.x).fit()
            max_p = max(regression.pvalues.values[start:])
            if max_p > p:
                for j in range(0, futures_num - i):
                    if regression.pvalues[j] == max_p:
                        self.x = self.x.drop(self.x.columns[[j]], axis=1)
        print(regression.summary())


class CustomLinearRegressionReg(CustomLinearRegression):
    def __init__(self, y, x, model_type=OLS):
        super().__init__(y, x, model_type)
        self.model = sm.OLS(self.y, self.x).fit_regularized(alpha=1., L1_wt=0.5, refit=True)

# class CustomRidgeLinearRegression(CustomLinearRegression):
#     def __init__(self, y, x, model_type=OLS):
#         super().__init__(y, x, model_type)
#         self.model = Ridge(alpha=0.1).fit(self.x, self.y)

# Отбор переменных для модели линейной регрессии

Отбор переменных для модели линейной регрессии - это процесс выбора наиболее важных и значимых независимых переменных, которые будут использоваться в модели для прогнозирования зависимой переменной. Это важный шаг в построении модели, так как использование неподходящих переменных может привести к низкой точности прогнозов и низкому качеству модели.

Существует несколько методов отбора переменных для модели линейной регрессии, включая:

- Метод пошагового отбора переменных - это метод, который включает или исключает независимые переменные в модель пошагово на основе их значимости. На каждом шаге происходит выбор переменной, которая наиболее улучшает качество модели.

- Метод отбора на основе значимости - этот метод основан на анализе значимости каждой независимой переменной с помощью статистических тестов, таких как t-тест или F-тест. Переменные с низким уровнем значимости исключаются из модели.

- Метод регуляризации - это метод, который добавляет штрафы за большие значения коэффициентов в модель. Он позволяет автоматически отбирать переменные, которые наиболее важны для модели, и уменьшает вероятность переобучения.

- Метод отбора на основе информационных критериев - этот метод основан на использовании различных информационных критериев, таких как AIC (критерий Акаике) и BIC (критерий Шварца), чтобы выбрать модель с наименьшей ошибкой.

Выбор метода отбора переменных зависит от конкретной задачи и объема доступных данных. Важно также учитывать возможную мультиколлинеарность между независимыми переменными, что может привести к искажению результатов. Поэтому важно проводить анализ и выбор переменных внимательно и осознанно.

Сначала посмотрим на данные, которые мы будем использовать для построения модели линейной регрессии.

In [3156]:
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-03-31,0.130033,0.039618,0.054579,0.000000,0.047500,0.034666,0.021491,0.184191,0.079561,0.118353,0.163565,0.080355,0.057975,0.076748
1998-06-30,0.109734,0.000000,0.070230,0.059514,0.065188,0.038675,0.143734,0.186478,0.041587,0.033739,0.094979,0.053325,0.003105,0.087566
1998-09-30,0.093442,0.083492,0.014272,0.084480,0.138413,0.000000,0.066975,0.271012,0.122117,0.098499,0.071404,0.083589,0.121368,0.040029
1998-12-31,0.148381,0.004023,0.078214,0.037996,0.056501,0.029885,0.116663,0.350363,0.094388,0.110388,0.104413,0.121304,0.069230,0.097603
1999-03-31,0.000000,0.184426,0.126300,0.046120,0.057533,0.103761,0.150218,0.310105,0.098186,0.087528,0.156815,0.033079,0.100402,0.104183


Мы имеем дело с набором данных, содержащим 11 независимых переменных и 3 зависимых переменных. Переменные x1-x11 являются независимыми переменными, а переменные y1-y3 являются зависимыми переменными. Всего у нас 25 наблюдений. Разделим данные на независимые и зависимые переменные.

### Разделение данных на зависимые и независимые переменные

Зависимые переменные - это переменные, значения которых мы хотим предсказать. В нашем случае это переменные y1, y2 и y3.

In [3157]:
y1_test, y1_train = y_test['y1'], y_train['y1']
y2_test, y2_train = y_test['y2'], y_train['y2']
y3_test, y3_train = y_test['y3'], y_train['y3']

Независимые переменные - это переменные, значения которых мы используем для прогнозирования зависимых переменных. В нашем случае это переменные x1-x11.

In [3158]:
x1_test, x1_train = X_test['x1'], X_train['x1']
x2_test, x2_train = X_test['x2'], X_train['x2']
x3_test, x3_train = X_test['x3'], X_train['x3']
x4_test, x4_train = X_test['x4'], X_train['x4']
x5_test, x5_train = X_test['x5'], X_train['x5']
x6_test, x6_train = X_test['x6'], X_train['x6']
x7_test, x7_train = X_test['x7'], X_train['x7']
x8_test, x8_train = X_test['x8'], X_train['x8']
x9_test, x9_train = X_test['x9'], X_train['x9']
x10_test, x10_train = X_test['x10'], X_train['x10']
x11_test, x11_train = X_test['x11'], X_train['x11']

# Оценка предварительно составленных систем уравнений

Всего у нас три системы уравнений, которые мы можем использовать для прогнозирования зависимых переменных
- Система независимых уравнений;
- Система рекурсивных уравнений;
- Система одновременных уравнений.

#### Система независимых уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

#### Система рекурсивных уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon \\
\end{cases}
$$

#### Система одновременных уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

## Система независимых уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon $$

#### Исходная система уравнений

In [3159]:
fn1_nez = pd.concat([x1_train, x2_train], axis=1)
CustomLinearRegression(y1_train, fn1_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.928
Model:                            OLS   Adj. R-squared (uncentered):              0.926
Method:                 Least Squares   F-statistic:                              404.8
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.11e-36
Time:                        16:34:52   Log-Likelihood:                          53.884
No. Observations:                  65   AIC:                                     -103.8
Df Residuals:                      63   BIC:                                     -99.42
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3160]:
CustomLinearRegression(y1_train, fn1_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.925
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              785.1
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.22e-37
Time:                        16:34:52   Log-Likelihood:                          52.488
No. Observations:                  65   AIC:                                     -103.0
Df Residuals:                      64   BIC:                                     -100.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3161]:
final_fn1_nez = pd.concat([x1_train, x2_train], axis=1)
CustomLinearRegression(y1_train, final_fn1_nez).get_coefficients()

x1    0.494660
x2    0.130508
dtype: float64
$$\hat y_1 = 0.4947x_{1} + 0.1305x_{2}$$


$$\hat y_1 = 0.3800x_{1} + 0.2251x_{2}$$

In [3162]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1_train, y=y1_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x2_train, y=y1_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X2', row=1, col=2)

fig.update_layout(title_text="График рассеяния")
fig.show()

### Второе уравнение

$$y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon$$

#### Исходная система уравнений

In [3163]:
fn2_nez = pd.concat([x1_train, x2_train, x3_train, x4_train, x5_train, x6_train, x7_train, x8_train, x9_train], axis=1)
CustomLinearRegression(y2_train, fn2_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.988
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              519.4
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.41e-50
Time:                        16:34:52   Log-Likelihood:                          97.156
No. Observations:                  65   AIC:                                     -176.3
Df Residuals:                      56   BIC:                                     -156.7
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3164]:
CustomLinearRegression(y2_train, fn2_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              1560.
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    2.57e-58
Time:                        16:34:52   Log-Likelihood:                          93.923
No. Observations:                  65   AIC:                                     -181.8
Df Residuals:                      62   BIC:                                     -175.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3165]:
final_fn2_nez = pd.concat([x1_train, x6_train, x7_train], axis=1)
CustomLinearRegression(y2_train, final_fn2_nez).get_coefficients()

x1    0.248908
x6    0.710528
x7   -0.000700
dtype: float64
$$\hat y_2 = 0.2489x_{1} + 0.7105x_{6} + -0.0007x_{7}$$


$$\hat y_2 = 0.1463x_{1} + 0.7211x_{6} + 0.0471x_{7}$$

In [3166]:
fig = make_subplots(rows=1, cols=3)
fig.update_yaxes(title_text='Y2', row=1, col=1)

fig.add_trace(px.scatter(x=x1_train, y=y2_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x6_train, y=y2_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X6', row=1, col=2)

fig.add_trace(px.scatter(x=x7_train, y=y2_train).data[0], row=1, col=3)
fig.update_xaxes(title_text='X7', row=1, col=3)

fig.update_layout(title='Графики рассеяния')
fig.show()

### Третье уравнение

$$y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon$$

#### Исходная система уравнений

In [3167]:
fn3_nez = pd.concat(
    [x1_train, x2_train, x3_train, x4_train, x5_train, x6_train, x7_train, x8_train, x9_train, x10_train, x11_train],
    axis=1)
CustomLinearRegression(y3_train, fn3_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.993
Model:                            OLS   Adj. R-squared (uncentered):              0.991
Method:                 Least Squares   F-statistic:                              680.7
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    9.53e-54
Time:                        16:34:53   Log-Likelihood:                          113.82
No. Observations:                  65   AIC:                                     -205.6
Df Residuals:                      54   BIC:                                     -181.7
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3168]:
CustomLinearRegression(y3_train, fn3_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.992
Model:                            OLS   Adj. R-squared (uncentered):              0.991
Method:                 Least Squares   F-statistic:                              1848.
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    6.97e-63
Time:                        16:34:53   Log-Likelihood:                          109.47
No. Observations:                  65   AIC:                                     -210.9
Df Residuals:                      61   BIC:                                     -202.2
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3169]:
final_fn3_nez = pd.concat([x1_train, x3_train, x4_train, x5_train, x10_train], axis=1)
CustomLinearRegression(y3_train, final_fn3_nez).get_coefficients()

x1     0.301129
x3    -0.026232
x4     0.406089
x5     0.122128
x10    0.194387
dtype: float64
$$\hat y_3 = 0.3011x_{1} + -0.0262x_{3} + 0.4061x_{4} + 0.1221x_{5} + 0.1944x_{10}$$


$$\hat y_3 = 0.2652x_{1} + 0.1689x_{3} + 0.2582x_{4} - 0.0136x_{5} + 0.2652x_{10}$$

In [3170]:
fig = make_subplots(rows=2, cols=3)
fig.update_yaxes(title_text='Y3', row=1, col=1)
fig.update_yaxes(title_text='Y3', row=2, col=1)

fig.add_trace(px.scatter(x=x1_train, y=y3_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x3_train, y=y3_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X3', row=1, col=2)

fig.add_trace(px.scatter(x=x4_train, y=y3_train).data[0], row=1, col=3)
fig.update_xaxes(title_text='X4', row=1, col=3)

fig.add_trace(px.scatter(x=x5_train, y=y3_train).data[0], row=2, col=1)
fig.update_xaxes(title_text='X5', row=2, col=1)

fig.add_trace(px.scatter(x=x10_train, y=y3_train).data[0], row=2, col=2)
fig.update_xaxes(title_text='X10', row=2, col=2)

fig.update_layout(height=800, title='Графики рассеяния')
fig.show()

## Система рекурсивных уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon$$

#### Исходная система уравнений

In [3171]:
fn1_rec = pd.concat([x1_train, x2_train], axis=1)
CustomLinearRegression(y1_train, fn1_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.928
Model:                            OLS   Adj. R-squared (uncentered):              0.926
Method:                 Least Squares   F-statistic:                              404.8
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.11e-36
Time:                        16:34:53   Log-Likelihood:                          53.884
No. Observations:                  65   AIC:                                     -103.8
Df Residuals:                      63   BIC:                                     -99.42
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3172]:
CustomLinearRegression(y1_train, fn1_rec).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.925
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              785.1
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.22e-37
Time:                        16:34:53   Log-Likelihood:                          52.488
No. Observations:                  65   AIC:                                     -103.0
Df Residuals:                      64   BIC:                                     -100.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3173]:
final_fn1_rec = pd.concat([x1_train, x2_train], axis=1)
CustomLinearRegression(y1_train, final_fn1_rec).get_coefficients()

x1    0.494660
x2    0.130508
dtype: float64
$$\hat y_1 = 0.4947x_{1} + 0.1305x_{2}$$


$$\hat y_1 = 0.3800x_{1} + 0.2251x_{2}$$

### Второе уравнение

$$y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon$$

#### Исходная система уравнений

In [3174]:
fn2_rec = pd.concat(
    [y1_train, x1_train, x2_train, x3_train, x4_train, x5_train, x6_train, x7_train, x8_train, x9_train], axis=1)
CustomLinearRegression(y2_train, fn2_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.988
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              459.3
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    3.29e-49
Time:                        16:34:53   Log-Likelihood:                          97.169
No. Observations:                  65   AIC:                                     -174.3
Df Residuals:                      55   BIC:                                     -152.6
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3175]:
CustomLinearRegression(y2_train, fn2_rec).backward_elimination(start=1)

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              1153.
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    1.06e-56
Time:                        16:34:53   Log-Likelihood:                          93.969
No. Observations:                  65   AIC:                                     -179.9
Df Residuals:                      61   BIC:                                     -171.2
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3176]:
final_fn2_rec = pd.concat([y1_train, x1_train, x3_train, x6_train], axis=1)
CustomLinearRegression(y2_train, final_fn2_rec).get_coefficients()

y1    0.256649
x1    0.345740
x3   -0.121790
x6    0.543714
dtype: float64
$$\hat y_2 = 0.2566y_{1} + 0.3457x_{1} + -0.1218x_{3} + 0.5437x_{6}$$


$$\hat y_2 = 0.4364y_{1} + 0.1611x_{1} - 0.0443x_{3} + 0.4761x_{6}$$

In [3177]:
fig = make_subplots(rows=2, cols=2)
fig.update_yaxes(title_text='Y2', row=1, col=1)
fig.update_yaxes(title_text='Y2', row=2, col=1)

fig.add_trace(px.scatter(x=y1_train, y=y2_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1_train, y=y2_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X1', row=1, col=2)

fig.add_trace(px.scatter(x=x3_train, y=y2_train).data[0], row=2, col=1)
fig.update_xaxes(title_text='X3', row=1, col=3)

fig.add_trace(px.scatter(x=x6_train, y=y2_train).data[0], row=2, col=2)
fig.update_xaxes(title_text='X6', row=2, col=1)

fig.update_layout(height=800, title='Графики рассеяния')
fig.show()

### Третье уравнение

$$y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon$$

#### Исходная система уравнений

In [3178]:
fn3_rec = pd.concat([y1_train, y2_train, x11_train], axis=1)
CustomLinearRegression(y3_train, fn3_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              1785.
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    4.18e-60
Time:                        16:34:53   Log-Likelihood:                          98.571
No. Observations:                  65   AIC:                                     -191.1
Df Residuals:                      62   BIC:                                     -184.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3179]:
CustomLinearRegression(y3_train, fn3_rec).backward_elimination(start=0)

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              1785.
Date:                Tue, 25 Apr 2023   Prob (F-statistic):                    4.18e-60
Time:                        16:34:53   Log-Likelihood:                          98.571
No. Observations:                  65   AIC:                                     -191.1
Df Residuals:                      62   BIC:                                     -184.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [3180]:
final_fn3_rec = pd.concat([y2_train], axis=1)
CustomLinearRegression(y3_train, final_fn3_rec).get_coefficients()

y2    0.98608
dtype: float64
$$\hat y_3 = 0.9861y_{2}$$


$$\hat y_3 = 1.0253y_{2}$$

In [3181]:
fig = make_subplots(rows=1, cols=1)
fig.update_yaxes(title_text='Y3', row=1, col=1)

fig.add_trace(px.scatter(x=y2_train, y=y3_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.update_layout(height=500, title='Графики рассеяния')
fig.show()

## Система одновременных уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon$$

#### Исходная система уравнений

In [3182]:
fn1_odn = pd.concat([x1_train, x2_train], axis=1)
CustomLinearRegression(y1_train, fn1_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.735163
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y1   No. Observations:                   65
Model:                        Poisson   Df Residuals:                       63
Method:                           MLE   Df Model:                            1
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1953
Time:                        16:34:53   Log-Likelihood:                -47.786
converged:                       True   LL-Null:                       -39.978
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.4577      0.754     -1.933      0.053      -2.936       0.020
x2             0.4008      1.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3183]:
CustomLinearRegression(y1_train, fn1_odn, model_type=Poisson).backward_elimination()

Optimization terminated successfully.
         Current function value: 0.735163
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.735163
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.735163
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.736248
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y1   No. Observations:                   65
Model:                        Poisson   Df Residuals:                       64
Method:                           MLE   Df Model:                            0
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1971
Time:                        16:34:53   Log-Likelihood:                -47.856
converged:                       True   LL-Null:                       -39.978
Covariance Type:            nonrobust  

#### Конечная система уравнений и веса

In [3184]:
final_fn1_odn = pd.concat([x1_train], axis=1)
CustomLinearRegression(y1_train, final_fn1_odn, model_type=Poisson).get_coefficients()

Optimization terminated successfully.
         Current function value: 0.736248
         Iterations 6
x1   -1.200329
dtype: float64
$$\hat y_1 = -1.2003x_{1}$$


$$\hat y_1 = -1.4598x_{1}$$

In [3185]:
fig = make_subplots(rows=1, cols=1)
fig.update_yaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1_train, y=y1_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.update_layout(height=500, title='Графики рассеяния')
fig.show()

### Второе уравнение

$$y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\$$

#### Исходная система уравнений

In [3186]:
fn2_odn = pd.concat([y1_train, x3_train, x4_train, x5_train, x6_train, x7_train, x8_train, x9_train], axis=1)
CustomLinearRegression(y2_train, fn2_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.732795
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y2   No. Observations:                   65
Model:                        Poisson   Df Residuals:                       57
Method:                           MLE   Df Model:                            7
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                -0.04962
Time:                        16:34:54   Log-Likelihood:                -47.632
converged:                       True   LL-Null:                       -45.380
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
y1            -4.5902      2.996     -1.532      0.125     -10.462       1.282
x3            -0.2255      1.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3187]:
CustomLinearRegression(y2_train, fn2_odn, model_type=Poisson).backward_elimination(start=1)

Optimization terminated successfully.
         Current function value: 0.732795
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.732795
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.732795
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.732948
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.733238
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.736738
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.739026
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.742428
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.742428
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.742428
  

#### Конечная система уравнений и веса

In [3188]:
final_fn2_odn = pd.concat([y1_train, x5_train], axis=1)
CustomLinearRegression(y2_train, final_fn2_odn, model_type=Poisson).get_coefficients()

Optimization terminated successfully.
         Current function value: 0.776808
         Iterations 6
y1    0.933081
x5   -1.766946
dtype: float64
$$\hat y_2 = 0.9331y_{1} + -1.7669x_{5}$$


$$\hat y_2 = 1.2371y_{1} - 2.0381x_{5}$$

In [3189]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y2', row=1, col=1)

fig.add_trace(px.scatter(x=y1_train, y=y2_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x3_train, y=y2_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X3', row=1, col=2)

fig.update_layout(height=500, title='Графики рассеяния')
fig.show()

### Третье уравнение

$$y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon$$

#### Исходная система уравнений

In [3190]:
fn3_odn = pd.concat([y1_train, x10_train, x11_train], axis=1)
CustomLinearRegression(y3_train, fn3_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.785296
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y3   No. Observations:                   65
Model:                        Poisson   Df Residuals:                       62
Method:                           MLE   Df Model:                            2
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1192
Time:                        16:34:54   Log-Likelihood:                -51.044
converged:                       True   LL-Null:                       -45.608
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
y1            -5.3577      2.177     -2.462      0.014      -9.624      -1.092
x10            4.4364      2.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [3191]:
CustomLinearRegression(y3_train, fn3_odn, model_type=Poisson).backward_elimination(start=0)

Optimization terminated successfully.
         Current function value: 0.785296
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.785296
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.785296
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.785899
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.785899
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y3   No. Observations:                   65
Model:                        Poisson   Df Residuals:                       63
Method:                           MLE   Df Model:                            1
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1201
Time:                        16:34:54   Log-Likelihood:                -51.083
converged:      

#### Конечная система уравнений и веса

In [3192]:
final_fn3_odn = pd.concat([y1_train], axis=1)
CustomLinearRegression(y3_train, final_fn3_odn, model_type=Poisson).get_coefficients()

Optimization terminated successfully.
         Current function value: 0.819653
         Iterations 6
y1   -1.37694
dtype: float64
$$\hat y_3 = -1.3769y_{1}$$


$$\hat y_3 = -1.6450y_{1}$$

In [3193]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y3', row=1, col=1)

fig.add_trace(px.scatter(x=x10_train, y=y3_train).data[0], row=1, col=1)
fig.update_xaxes(title_text='X11', row=1, col=1)

fig.add_trace(px.scatter(x=x11_train, y=y3_train).data[0], row=1, col=2)
fig.update_xaxes(title_text='X11', row=1, col=2)

fig.update_layout(height=500, title='Графики рассеяния')
fig.show()

# Прогнозирование

### Система независимых уравнеий

$$
\begin{cases}
\hat y_1 = 0.3800x_{1} + 0.2251x_{2} \\
\hat y_2 = 0.1463x_{1} + 0.7211x_{6} + 0.0471x_{7} \\
\hat y_3 = 0.2652x_{1} + 0.1689x_{3} + 0.2582x_{4} - 0.0136x_{5} + 0.2652x_{10} \\
\end{cases}
$$

In [3194]:
final_fn1_nez.columns.values.tolist(), final_fn2_nez.columns.values.tolist(), final_fn3_nez.columns.values.tolist()

(['x1', 'x2'], ['x1', 'x6', 'x7'], ['x1', 'x3', 'x4', 'x5', 'x10'])

#### Первое уравнение

In [3195]:
CustomLinearRegression(y1_train, final_fn1_nez).predict(pd.concat([x1_test, x2_test], axis=1), y1_test)

Коэффициент детерминации (R^2): 70.4%
Cредняя ошибка аппроксимации (MAE): 7.2%
Cредняя квадратичная ошибка (MSE): 0.8%


#### Второе уравнение

In [3196]:
CustomLinearRegression(y2_train, final_fn2_nez).predict(pd.concat([x1_test, x6_test, x7_test], axis=1), y2_test)

Коэффициент детерминации (R^2): 87.0%
Cредняя ошибка аппроксимации (MAE): 6.6%
Cредняя квадратичная ошибка (MSE): 0.7%


#### Третье уравнение

In [3197]:
CustomLinearRegression(y3_train, final_fn3_nez).predict(pd.concat([x1_test, x3_test, x4_test, x5_test, x10_test], axis=1), y3_test)

Коэффициент детерминации (R^2): 95.8%
Cредняя ошибка аппроксимации (MAE): 3.5%
Cредняя квадратичная ошибка (MSE): 0.2%


### Система рекурсивных уравнеий

$$
\begin{cases}
\hat y_1 = 0.3800x_{1} + 0.2251x_{2} \\
\hat y_2 = 0.4364y_{1} + 0.1611x_{1} - 0.0443x_{3} + 0.4761x_{6} \\
\hat y_3 = 1.0253y_{2} \\
\end{cases}
$$

In [3198]:
final_fn1_rec.columns.values.tolist(), final_fn2_rec.columns.values.tolist(), final_fn3_rec.columns.values.tolist()

(['x1', 'x2'], ['y1', 'x1', 'x3', 'x6'], ['y2'])

#### Первое уравнение

In [3199]:
CustomLinearRegression(y1_train, final_fn1_rec).predict(pd.concat([x1_test, x2_test], axis=1), y1_test)

Коэффициент детерминации (R^2): 70.4%
Cредняя ошибка аппроксимации (MAE): 7.2%
Cредняя квадратичная ошибка (MSE): 0.8%


#### Второе уравнение

In [3200]:
CustomLinearRegression(y2_train, final_fn2_rec).predict(pd.concat([y1_test, x1_test, x3_test, x6_test], axis=1), y2_test)

Коэффициент детерминации (R^2): 87.7%
Cредняя ошибка аппроксимации (MAE): 6.6%
Cредняя квадратичная ошибка (MSE): 0.6%


#### Третье уравнение

In [3201]:
CustomLinearRegression(y3_train, final_fn3_rec).predict(pd.concat([y2_test], axis=1), y3_test)

Коэффициент детерминации (R^2): 88.8%
Cредняя ошибка аппроксимации (MAE): 6.7%
Cредняя квадратичная ошибка (MSE): 0.6%


### Система одновременных уравнеий

$$
\begin{cases}
\hat y_1 = -1.4598x_{1} \\
\hat y_2 = 1.2371y_{1} -2.0381x_{5} \\
\hat y_3 = -1.6450y_{1} \\
\end{cases}
$$

In [3202]:
final_fn1_odn.columns.values.tolist(), final_fn2_odn.columns.values.tolist(), final_fn3_odn.columns.values.tolist()

(['x1'], ['y1', 'x5'], ['y1'])

#### Первое уравнение

In [3203]:
CustomLinearRegression(y1_train, final_fn1_odn).predict(pd.concat([x1_test], axis=1), y1_test)

Коэффициент детерминации (R^2): 62.2%
Cредняя ошибка аппроксимации (MAE): 8.3%
Cредняя квадратичная ошибка (MSE): 1.0%


#### Второе уравнение

In [3204]:
CustomLinearRegression(y2_train, final_fn2_odn).predict(pd.concat([y1_test, x5_test], axis=1), y2_test)

Коэффициент детерминации (R^2): 79.4%
Cредняя ошибка аппроксимации (MAE): 7.7%
Cредняя квадратичная ошибка (MSE): 1.1%


#### Третье уравнение

In [3205]:
CustomLinearRegression(y3_train, final_fn3_odn).predict(pd.concat([y1_test], axis=1), y3_test)

Коэффициент детерминации (R^2): 81.4%
Cредняя ошибка аппроксимации (MAE): 8.5%
Cредняя квадратичная ошибка (MSE): 1.0%
